In [ ]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CANDIDATE,0,0,0,0,0.259820,4.040000e-07,-4.040000e-07,131.850610,0.001370,...,-62,4.736,0.028,-0.035,0.526,0.035,-0.035,281.11646,43.282440,15.174
1,FALSE POSITIVE,0,1,0,0,0.299698,1.910000e-07,-1.910000e-07,132.017121,0.000528,...,-154,4.547,0.058,-0.071,0.782,0.090,-0.074,296.96381,50.745380,14.828
2,CANDIDATE,0,0,0,0,0.306702,7.190000e-07,-7.190000e-07,131.512160,0.002070,...,-184,4.512,0.095,-0.085,0.786,0.110,-0.099,289.82599,43.725231,15.229
3,FALSE POSITIVE,0,1,0,0,0.306938,3.330000e-07,-3.330000e-07,131.635518,0.000900,...,-138,4.580,0.044,-0.061,0.741,0.081,-0.066,287.73572,42.823421,14.409
4,FALSE POSITIVE,0,1,0,0,0.328687,4.620000e-07,-4.620000e-07,132.771460,0.001260,...,-160,4.535,0.048,-0.143,0.847,0.181,-0.077,288.41684,47.731091,15.316


# Select your features (columns)

In [5]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec']]

# Create a Train Test Split

Use `koi_disposition` for the y values

In [6]:
y = pd.get_dummies(df["koi_disposition"])
X = selected_features
print(X.shape, y.shape)

(6991, 4) (6991, 3)


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec
6122,0,0,0,0
6370,1,0,0,0
2879,0,0,0,0
107,0,0,1,0
29,0,0,0,0


In [14]:
# NEW Cell:  Create model usin linear regression
from sklearn.linear_model import LinearRegression
model = LinearRegression()

In [24]:
# NEW Cell:  Fit the model to the training date and calculate the scores for the training and testing data
model.fit(X_train, y_train)
training_score = model.score(X_train, y_train)
testing_score = model.score(X_test, y_test)

print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")

Training Score: 0.43333543908993444
Testing Score: 0.4297394197263542


In [ ]:
# NEW Cell:


In [ ]:
# NEW Cell:

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [12]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler

X_scaler = MinMaxScaler().fit(X_train)
y_scaler = MinMaxScaler().fit(y_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

# Train the Model



In [17]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.43333543908993444
Testing Data Score: 0.4297394197263542


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [ ]:
# Create the GridSearchCV model

In [ ]:
# Train the model with GridSearch

In [ ]:
print(grid2.best_params_)
print(grid2.best_score_)

# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'your_name.sav'
joblib.dump(your_model, filename)